In [1]:
class Solution:
  def __init__(self, knapsack):
    self.knapsack = knapsack.copy()

    self.w, self.v = self.get_knapsack_value(knapsack)

  def get_knapsack_value(self, knapsack):
    w = 0
    v = 0

    for item in knapsack:
      w += item[1]
      v += item[2]

    return w, v

In [133]:
class TabuSearch:
  def __init__(self, items, max_weight, tabu_size=15):
    self.items = items
    self.knapsack = []
    self.current_weight = 0

    self.max_weight = max_weight

    self.tabu_size = tabu_size
    self.tabu_list = deque()

  
  def search(self, max_iter=10000, max_iter_no_increment=500):
    self.generate_solution()  # gera uma solução aleatória
    best_solution = Solution(self.knapsack)

    print("Solução inicial: ", self.knapsack)

    iter_no_increment = 0

    for i in range(max_iter):
      n = self.get_neighbouring_solutions() # gera os vizinhos permitidos (peso <= limite)

      self.remove_tabu_solutions(n, best_solution) # remove os movimentos tabu dos vizinhos (considera aspiração)
      move = self.update_solution(n[0])

      if self.get_current_value() > best_solution.v:
        iter_no_increment = 0
        best_solution = Solution(self.knapsack)

        print("\nNova melhor solução: ", self.knapsack, "\tCom o movimento: ", move)
      else:
        iter_no_increment += 1

      if iter_no_increment > max_iter_no_increment:
        return best_solution

      self.update_tabu_list(move)
    
    return best_solution


  def get_current_value(self):
    value = 0
    for item in self.knapsack:
      value += item[2]

    return value


  def update_solution(self, n):
    if n[0] is 'a':
      self.add_item(n[1][0])
      return ('a', n[1][0]) # movimento feito
    else:
      self.remove_item(n[1][0])
      return ('r', n[1][0])

  
  def update_tabu_list(self, move):
    if len(self.tabu_list) == self.tabu_size:
      self.tabu_list.popleft()

    self.tabu_list.append(move)


  def add_item(self, i):
    j = [x[0] for x in self.items].index(i) # encontrar o indice na lista de items (o indice passado é o inicial)

    item = self.items[j]

    if self.check_add(item):
      self.knapsack.append(self.items.pop(j))
      self.current_weight += item[1]
      return True

    return False
    
  def remove_item(self, i):
    j = [x[0] for x in self.knapsack].index(i)

    self.current_weight -= self.knapsack[j][1]
    self.items.append(self.knapsack.pop(j))


  def generate_solution(self):
    rng = random.sample(range(len(self.items)), len(self.items))

    for i in rng:
      if not self.add_item(i):
        return 

  
  def get_neighbouring_solutions(self):
    return self.get_add_neighbors() + self.get_remove_neighbors()

  def get_add_neighbors(self):
    n = []

    for item in self.items:
      if self.check_add(item):
        n.append(('a', item))

    n.sort(key=lambda x: x[1][2], reverse=True)
    
    return n

  def get_remove_neighbors(self):
    n = []

    for item in self.knapsack:
      n.append(('r', item))

    n.sort(key=lambda x: x[1][2])

    return n


  def check_add(self, item):
    if item[1] + self.current_weight <= self.max_weight:
      return True

    return False


  def remove_tabu_solutions(self, l, best_solution):
    current_val = self.get_current_value()
    for item in l:
      if (item[0], item[1][0]) in self.tabu_list:
        if item[0] is 'a' and item[1][2] + current_val <= best_solution.v:
          l.remove(item)

In [134]:
import random
from collections import deque

In [135]:
w = [63, 21,  2, 32, 13, 80, 19, 37, 56, 41, 14,  8, 32, 42,  7]
v = [13,  2, 20, 10,  7, 14,  7,  2,  2,  4, 16, 17, 17,  3, 21]

max_weight = 275

In [189]:
wv = list(zip(range(len(w)), w, v))

t = TabuSearch(wv, max_weight, tabu_size=10)

In [190]:
# cada item = (indice inicial, peso, valor)

solution = t.search()

Solução inicial:  [(3, 32, 10), (13, 42, 3), (0, 63, 13), (5, 80, 14), (4, 13, 7), (7, 37, 2), (11, 8, 17)]

Nova melhor solução:  [(3, 32, 10), (13, 42, 3), (0, 63, 13), (5, 80, 14), (4, 13, 7), (11, 8, 17), (14, 7, 21)] 	Com o movimento:  ('a', 14)

Nova melhor solução:  [(3, 32, 10), (13, 42, 3), (0, 63, 13), (5, 80, 14), (4, 13, 7), (11, 8, 17), (14, 7, 21), (2, 2, 20)] 	Com o movimento:  ('a', 2)

Nova melhor solução:  [(3, 32, 10), (13, 42, 3), (0, 63, 13), (5, 80, 14), (4, 13, 7), (11, 8, 17), (14, 7, 21), (2, 2, 20), (10, 14, 16)] 	Com o movimento:  ('a', 10)

Nova melhor solução:  [(3, 32, 10), (0, 63, 13), (5, 80, 14), (4, 13, 7), (11, 8, 17), (14, 7, 21), (2, 2, 20), (10, 14, 16), (12, 32, 17)] 	Com o movimento:  ('a', 12)

Nova melhor solução:  [(3, 32, 10), (0, 63, 13), (5, 80, 14), (4, 13, 7), (11, 8, 17), (14, 7, 21), (2, 2, 20), (10, 14, 16), (12, 32, 17), (6, 19, 7)] 	Com o movimento:  ('a', 6)


In [191]:
solution.knapsack

[(3, 32, 10),
 (0, 63, 13),
 (5, 80, 14),
 (4, 13, 7),
 (11, 8, 17),
 (14, 7, 21),
 (2, 2, 20),
 (10, 14, 16),
 (12, 32, 17),
 (6, 19, 7)]

In [192]:
solution.w

270

In [193]:
solution.v

142